From episcanpy use epi.tl.find_genes to find the peaks

# File opening(skip)

In [3]:
#edit based on your file path

file_path1 = "/Users/gajaj/Downloads/H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_raw_feature_bc_matrix.tar.gz"
output_dir1 = "/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome"

In [4]:
with tarfile.open(file_path1, "r") as tar:
    tar.extractall(path=output_dir1)
    print("Files extracted successfully")

Files extracted successfully


In [5]:
#edit your file paths
input_file_barcodes1 = "Multiome/raw_feature_bc_matrix/barcodes.tsv.gz"
input_file_features1 = "Multiome/raw_feature_bc_matrix/features.tsv.gz"
input_file_matrix1 = "Multiome/raw_feature_bc_matrix/matrix.mtx.gz"

In [6]:
#edit your file paths
output_file_barcodes1 = "Multiome/raw_feature_bc_matrix/barcodes.tsv"
output_file_features1 = "Multiome/raw_feature_bc_matrix/features.tsv"
output_file_matrix1 = "Multiome/raw_feature_bc_matrix/matrix.mtx"


In [7]:
import gzip

with gzip.open(input_file_barcodes1, "rt") as gz_file:  
    with open(output_file_barcodes1, "w") as out_file:
        out_file.write(gz_file.read())

In [8]:
with gzip.open(input_file_features1, "rt") as gz_file:  
    with open(output_file_features1, "w") as out_file:
        out_file.write(gz_file.read())

In [9]:
with gzip.open(input_file_matrix1, "rt") as gz_file:  
    with open(output_file_matrix1, "w") as out_file:
        out_file.write(gz_file.read())

# Overlaps for batch 1 data

In [1]:
import pandas as pd
import pandas as pd
import pyranges as pr

### Load the preprocessed data for genes

In [74]:
rna_batch1_test = pd.read_csv('../data/splits/batch2_test_filtered.tsv', sep='\t')
rna_batch1_val = pd.read_csv('../data/splits/batch2_val_filtered.tsv', sep='\t')
rna_batch1_train = pd.read_csv('../data/splits/batch2_training_filtered.tsv', sep='\t')

In [77]:
# Merge all the datasets
# List of DataFrames
df_list = [rna_batch1_test, rna_batch1_train, rna_batch1_val]

# Concatenate vertically (stack on top of each other)
rna_df = pd.concat(df_list, ignore_index=True)

# Keep only first two columns
rna_df = rna_df[['barcode', 'gene_id']]

print(rna_df)

                  barcode          gene_id
0      AACAAAGGTGTAACCA-2  ENSG00000187583
1      AACAAAGGTGTAACCA-2  ENSG00000130775
2      AACAAAGGTGTAACCA-2  ENSG00000117395
3      AACAAAGGTGTAACCA-2  ENSG00000269113
4      AACAAAGGTGTAACCA-2  ENSG00000186094
...                   ...              ...
34422  TTTGTGGCATGAATAG-2  ENSG00000130254
34423  TTTGTGGCATGAATAG-2  ENSG00000080573
34424  TTTGTGGCATGAATAG-2  ENSG00000167615
34425  TTTGTGGCATGAATAG-2  ENSG00000225377
34426  TTTGTGGCATGAATAG-2  ENSG00000198712

[34427 rows x 2 columns]


### Load the preprocessed atac data

In [60]:
atac_batch1 = pd.read_csv('../data/preprocessing/atac_batch2.tsv', sep='\t')

In [61]:
atac_df = atac_batch1

# Filter for nonzero expression
atac_df = atac_df[atac_df['opennes'] != 0]

# Keep only first two columns
atac_df = atac_df[['barcode', 'peak region']]

print(atac_df)

                      barcode               peak region
24         AAACAGCCAAACGCGA-2       chr1:999931-1000736
26         AAACAGCCAAACGCGA-2      chr1:1006265-1007161
50         AAACAGCCAAACGCGA-2      chr1:1231651-1232570
51         AAACAGCCAAACGCGA-2      chr1:1243919-1244727
96         AAACAGCCAAACGCGA-2      chr1:1674708-1675636
...                       ...                       ...
394602888  TTTGTTGGTTAGCTGA-2  chrX:135051785-135052578
394602900  TTTGTTGGTTAGCTGA-2  chrX:136146969-136147876
394603039  TTTGTTGGTTAGCTGA-2  chrX:153708167-153709083
394603049  TTTGTTGGTTAGCTGA-2  chrX:153934635-153935533
394603188  TTTGTTGGTTAGCTGA-2    GL000195.1:74770-75538

[15951195 rows x 2 columns]


In [62]:
# Split the column peak regions and rename chromosomes

# Step 1: Split 'peak region' into Chromosome and Positions
atac_df[['Chromosome', 'Positions']] = atac_df['peak region'].str.split(':', expand=True)

# Step 2: Strip 'chr' from Chromosome values
atac_df['Chromosome'] = atac_df['Chromosome'].str.lstrip('chr')

# Step 3: Split Positions into Start and End
atac_df[['Start', 'End']] = atac_df['Positions'].str.split('-', expand=True)

# Step 4: Convert Start and End to integers
atac_df['Start'] = atac_df['Start'].astype(int)
atac_df['End'] = atac_df['End'].astype(int)

# Step 5: Drop the original 'peak region' and 'Positions' columns
atac_df = atac_df.drop(columns=['peak region', 'Positions'])

# Display the transformed DataFrame
print(atac_df)


                      barcode  Chromosome      Start        End
24         AAACAGCCAAACGCGA-2           1     999931    1000736
26         AAACAGCCAAACGCGA-2           1    1006265    1007161
50         AAACAGCCAAACGCGA-2           1    1231651    1232570
51         AAACAGCCAAACGCGA-2           1    1243919    1244727
96         AAACAGCCAAACGCGA-2           1    1674708    1675636
...                       ...         ...        ...        ...
394602888  TTTGTTGGTTAGCTGA-2           X  135051785  135052578
394602900  TTTGTTGGTTAGCTGA-2           X  136146969  136147876
394603039  TTTGTTGGTTAGCTGA-2           X  153708167  153709083
394603049  TTTGTTGGTTAGCTGA-2           X  153934635  153935533
394603188  TTTGTTGGTTAGCTGA-2  GL000195.1      74770      75538

[15951195 rows x 4 columns]


### Load the dataset of gene positions created in `get_gene_position.r`

In [63]:
# Load gene regions dataset
gene_position_df = pd.read_csv("../data/gene_positions_and_overlaps/gene_positions.csv")

In [64]:
gene_position_df.columns = ["seqnames", "start", "end", "gene_id"]

In [65]:
# Rename the columns
gene_position_df = gene_position_df.rename(columns={
    "seqnames": "Chromosome",  # Rename seqnames to Chromosome
    "start": "Start",          # Rename start to Start
    "end": "End"               # Rename end to End
})
gene_position_df

,Chromosome,Start,End,gene_id
0,1,3069168,3438621,ENSG00000142611
1,1,5301928,5307394,ENSG00000284616
2,1,2403964,2413797,ENSG00000157911
3,1,5492978,5494674,ENSG00000260972
4,1,10054445,10054781,ENSG00000224340
...,...,...,...,...
78927,KI270717.1,15119,17798,ENSG00000307722
78928,KI270717.1,24947,37269,ENSG00000310401
78929,KI270720.1,3633,31375,ENSG00000302039
78930,KI270718.1,6913,19848,ENSG00000309831


### Merge the dataset of positions with rna

In [66]:
# Merge on 'gene_id', keeping only matches (inner join)
rna_positions_df = pd.merge(rna_df, gene_position_df, on="gene_id", how="inner")

# Display result
print(rna_positions_df)


                  barcode          gene_id Chromosome     Start       End
0      AACAAAGGTGTAACCA-2  ENSG00000187583          1    966482    975865
1      AACAAAGGTGTAACCA-2  ENSG00000130775          1  27872543  27886685
2      AACAAAGGTGTAACCA-2  ENSG00000117395          1  43164175  43270936
3      AACAAAGGTGTAACCA-2  ENSG00000269113          1  47760528  47997385
4      AACAAAGGTGTAACCA-2  ENSG00000186094          1  48532854  50023954
...                   ...              ...        ...       ...       ...
33752  TTTGTGGCATGAATAG-2  ENSG00000130254         19   5586981   5624046
33753  TTTGTGGCATGAATAG-2  ENSG00000080573         19   9959561  10010504
33754  TTTGTGGCATGAATAG-2  ENSG00000167615         19  54449190  54462016
33755  TTTGTGGCATGAATAG-2  ENSG00000225377         20    316860    348490
33756  TTTGTGGCATGAATAG-2  ENSG00000198712         MT      7586      8269

[33757 rows x 5 columns]


Create a buffer on gene regions

In [67]:
rna_positions_df['Start'] = rna_positions_df['Start'] - 2000
rna_positions_df['End'] = rna_positions_df['End'] + 2000
rna_positions_df

,barcode,gene_id,Chromosome,Start,End
0,AACAAAGGTGTAACCA-2,ENSG00000187583,1,964482,977865
1,AACAAAGGTGTAACCA-2,ENSG00000130775,1,27870543,27888685
2,AACAAAGGTGTAACCA-2,ENSG00000117395,1,43162175,43272936
3,AACAAAGGTGTAACCA-2,ENSG00000269113,1,47758528,47999385
4,AACAAAGGTGTAACCA-2,ENSG00000186094,1,48530854,50025954
...,...,...,...,...,...
33752,TTTGTGGCATGAATAG-2,ENSG00000130254,19,5584981,5626046
33753,TTTGTGGCATGAATAG-2,ENSG00000080573,19,9957561,10012504
33754,TTTGTGGCATGAATAG-2,ENSG00000167615,19,54447190,54464016
33755,TTTGTGGCATGAATAG-2,ENSG00000225377,20,314860,350490


### Merge using PyRanges

In [68]:
# Convert 'Start' and 'End' to integers
rna_positions_df["Start"] = rna_positions_df["Start"].astype(int)
rna_positions_df["End"] = rna_positions_df["End"].astype(int)

atac_df["Start"] = atac_df["Start"].astype(int)
atac_df["End"] = atac_df["End"].astype(int)

# Verify column types
print(rna_positions_df.dtypes)
print(atac_df.dtypes)


barcode       object
gene_id       object
Chromosome    object
Start          int64
End            int64
dtype: object
barcode       object
Chromosome    object
Start          int64
End            int64
dtype: object


In [69]:
# Convert dataframes to PyRanges format
rna_ranges = pr.PyRanges(rna_positions_df)
atac_ranges = pr.PyRanges(atac_df)

In [14]:
print(rna_ranges)
print(atac_ranges)

+--------------------+-----------------+--------------+-----------+-----------+
| barcode            | gene_id         | Chromosome   | Start     | End       |
| (object)           | (object)        | (category)   | (int64)   | (int64)   |
|--------------------+-----------------+--------------+-----------+-----------|
| AAACCGAAGGCGCATC-1 | ENSG00000269113 | 1            | 47758528  | 47999385  |
| AAACCGAAGGCGCATC-1 | ENSG00000231252 | 1            | 60657631  | 60870041  |
| AAACCGAAGGCGCATC-1 | ENSG00000188641 | 1            | 97075743  | 97997000  |
| AAACCGAAGGCGCATC-1 | ENSG00000265972 | 1            | 145990435 | 145998579 |
| ...                | ...             | ...          | ...       | ...       |
| TGATGAACAAGGCCAA-1 | ENSG00000176728 | Y            | 18770706  | 19079555  |
| TGGCCTTTCTTAATGG-1 | ENSG00000114374 | Y            | 12535650  | 12862839  |
| TGGCCTTTCTTAATGG-1 | ENSG00000183878 | Y            | 13232577  | 13482673  |
| TTCTTAGGTCACACCC-1 | ENSG00000183878 |

In [70]:
# Perform overlap-based merging
merged_df = atac_ranges.join(rna_ranges).df

In [71]:
# Keep only rows where barcode matches barcode_b
merged_df = merged_df[merged_df["barcode"] == merged_df["barcode_b"]]
merged_df = merged_df.drop(columns=['barcode_b'])

# Rename the columns
merged_df = merged_df.rename(columns={
    "Start": "Start peak",          
    "End": "End peak",
    "Start_b": "Start gene",          
    "End_b": "End gene"                  
})

In [72]:
print(merged_df)

                     barcode Chromosome  Start peak   End peak  \
12392     AAAGCACCAACAGCCT-2          1    47894501   47895341   
12656     AAAGCACCAACAGCCT-2          1   145996332  145997192   
12749     AAAGCACCAACAGCCT-2          1   245370903  245371717   
12816     AAAGCACCAACAGCCT-2          1   245583275  245584148   
12883     AAAGCACCAACAGCCT-2          1   245585086  245585913   
...                      ...        ...         ...        ...   
69241125  TTGTTTGGTTTGGTTC-2          X    56099086   56099949   
69242846  TTTACGCGTCCCGAAG-2          X    57286358   57287219   
69243045  TTTACGCGTCCCGAAG-2          X   139126243  139127132   
69244368  TTTATGGAGCGGTTAT-2          X   132217837  132218484   
69248342  TTTCTTGCAAACCTTG-2          X   139204768  139205644   

                  gene_id  Start gene   End gene  
12392     ENSG00000269113    47758528   47999385  
12656     ENSG00000265972   145990435  145998579  
12749     ENSG00000162849   245152985  245711432  
128

### Create a tsv file

In [73]:
# Save as a TSV file
merged_df.to_csv("../data/gene_positions_and_overlaps/overlaps_batch2.tsv", sep="\t", index=False)

# Confirm file creation
print("TSV file created successfully!")

TSV file created successfully!
